![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Financial Word and Sentence Embeddings

# Finance Word and Sentence Embeddings visualization using PCA (Principal Component Analysis)

Modern NLP models work with a numerical representation of texts and their menaning. For token classification problems (inferring a class for a token, for example Name Entity Recognition) Word Embeddings are required. For sentences, paragraph, document classification - we use Sentence Embeddings.

In this notebook, we got token embeddings using Spark NLP Finance Word Embeddings(**bert_embeddings_sec_bert_base**) and using these token embeddings we got sentence embeddings by sparknlp annotator SentenceEmbeddings to get those numerical representations of the semantics of the texts. The result is a 768 embeddings matrix, impossible to process by the human eye.

There are many techniques we can use to visualize those embeddings. We are using one of them - Principal Component Analysis, a dimensionality reduction process, carried out by Spark MLLib. Both embeddings have 768 dimensions, so we will reduced this dimensions from **768** to **3** (X, Y, Z) and will use a color for the word / sentence legend.

## Setup

In [ ]:
%pip install plotly
%pip install -q tensorflow==2.7.0
%pip install -q tensorflow-addons

In [ ]:
from johnsnowlabs import *

import json
import os

print("Spark NLP Version :", sparknlp.version())

spark = start_spark()

# Get sample text

In [ ]:
# Downloading sample datasets.
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings_JSL/Finance/data/finance_pca_samples.csv

In [ ]:
import pandas as pd

df = pd.read_csv("finance_pca_samples.csv")

In [ ]:
# Create spark dataframe
sdf = spark.createDataFrame(df)
sdf.show()

# Pipeline with Spark NLP and Spark MLLIB

In [ ]:
# We defined a generic pipeline for word and sentence embeddings

def generic_pipeline():
    document_assembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

    tokenizer = nlp.Tokenizer()\
      .setInputCols("document")\
      .setOutputCol("token")

    word_embeddings = nlp.BertEmbeddings.pretrained("bert_embeddings_sec_bert_base","en")\
      .setInputCols(["document", "token"])\
      .setOutputCol("word_embeddings")

    pipeline = Pipeline(stages = [
      document_assembler,
      tokenizer,
      word_embeddings
    ])

    return pipeline



## Sentence Embeddings

In [ ]:
embeddings_sentence = nlp.SentenceEmbeddings()\
    .setInputCols(["document", "word_embeddings"])\
    .setOutputCol("sentence_embeddings")\
    .setPoolingStrategy("AVERAGE")
# We used sparknlp SentenceEmbeddings anootator to get each sentence embeddings from token embeddings

# Custom transform to retrieve the numerical embeddings from Spark NLP and pass it to Spark MLLib

In [ ]:
# This class extracts the embeddings from the Spark NLP Annotation object
# from pyspark import ml as ML
class EmbeddingsUDF(
    Transformer, ML.param.shared.HasInputCol, ML.param.shared.HasOutputCol,
    ML.util.DefaultParamsReadable, ML.util.DefaultParamsWritable
):
    @keyword_only
    def __init__(self):
        super(EmbeddingsUDF, self).__init__()

        def _sum(r):
            result = 0.0
            for e in r:
                result += e
            return result

        self.udfs = {
            'convertToVectorUDF': F.udf(lambda vs: ML.linalg.Vectors.dense(vs), ML.linalg.VectorUDT()),
            'sumUDF': F.udf(lambda r: _sum(r), T.FloatType())
        }

    def _transform(self, dataset):

        results = dataset.select(
            "*", F.explode("sentence_embeddings.embeddings").alias("embeddings")
        )
        results = results.withColumn(
            "features",
            self.udfs['convertToVectorUDF'](F.col("embeddings"))
        )
        results = results.withColumn(
            "emb_sum",
            self.udfs['sumUDF'](F.col("embeddings"))
        )
        # Remove those with embeddings all zeroes (so we can calculate cosine distance)
        results = results.where(F.col("emb_sum")!=0.0)

        return results

In [ ]:
embeddings_for_pca = EmbeddingsUDF()

In [ ]:
DIMENSIONS  = 3

In [ ]:
pca = ML.feature.PCA(k=DIMENSIONS, inputCol="features", outputCol="pca_features")

### Full Spark NLP + Spark MLLib pipeline

In [ ]:
# We did all process in one pipeline
pipeline = Pipeline().setStages([generic_pipeline(), embeddings_sentence, embeddings_for_pca, pca])

In [ ]:
model = pipeline.fit(sdf)

In [ ]:
result = model.transform(sdf)

In [ ]:
result.select('pca_features', 'label').show(truncate=False)

In [ ]:
df = result.select('pca_features', 'label').toPandas()

df
# As you see, dimension values are inside a list

In [ ]:
# We extract the dimension values out off the list

df["x"] = df["pca_features"].apply(lambda x: x[0])

df["y"] = df["pca_features"].apply(lambda x: x[1])

df["z"] = df["pca_features"].apply(lambda x: x[2])

df = df[["x", "y", "z", "label"]]

df

In [ ]:
import plotly.express as px

fig = px.scatter_3d(df, x = 'x', y = 'y', z = 'z', color = 'label', width=800, height=600)

fig.show()

### Word Embeddings

We can also visualize the semantics of words, instead of full texts, by using Word Embeddings. We will add a Tokenizer and a WordEmbeddings model to get those embeddings, and them apply PCA as before. Firstly we splitted the pipeline in two to get all token embeddings

In [ ]:
model = generic_pipeline().fit(sdf)

In [ ]:
result = model.transform(sdf)

In [ ]:
result_df = result.select("label", F.explode(F.arrays_zip(result.token.result, result.word_embeddings.embeddings)).alias("cols"))\
      .select(F.expr("cols['0']").alias("token"),
              "label",
              F.expr("cols['1']").alias("embeddings"))

result_df.show(truncate = 80)


In [ ]:
# Here we defined inheritance class from that defined previously EmbeddingsUDF class
class WordEmbeddingsUDF(EmbeddingsUDF):    
    def _transform(self, dataset):
        
        results = dataset.select('token', 'label', 'embeddings') # We changed this line because our embedding cloumn is already exploded

        results = results.withColumn(
            "features",
            self.udfs['convertToVectorUDF'](F.col("embeddings"))
        )
        results = results.withColumn(
            "emb_sum",
            self.udfs['sumUDF'](F.col("embeddings"))
        )
        # Remove those with embeddings all zeroes (so we can calculate cosine distance)
        results = results.where(F.col("emb_sum")!=0.0)

        return results

In [ ]:
embeddings_for_pca = WordEmbeddingsUDF()

In [ ]:
DIMENSIONS  = 3

In [ ]:
pca = ML.feature.PCA(k=DIMENSIONS, inputCol="features", outputCol="pca_features")

### Full Spark NLP + Spark MLLib pipeline

In [ ]:
# We run the second part of the pipeline. Here 768 dimensions is reduced to 3 dimensions

pipeline = Pipeline().setStages([embeddings_for_pca, pca])


In [ ]:
model = pipeline.fit(result_df)

In [ ]:
result = model.transform(result_df)

In [ ]:
result.select("token", "label", "pca_features").show(truncate = 60)

In [ ]:
df = result.select('token', 'label', 'pca_features').toPandas()

df

In [ ]:
df["x"] = df["pca_features"].apply(lambda x: x[0])

df["y"] = df["pca_features"].apply(lambda x: x[1])

df["z"] = df["pca_features"].apply(lambda x: x[2])

df = df[["token", "label", "x", "y", "z"]]

df

In [ ]:
import plotly.express as px

fig = px.scatter_3d(df, x = 'x', y = 'y', z = 'z', color = "label", width=1000, height = 800, hover_data = ["token", "label"])

fig.show()